# Transformée de Fourier à court terme (dite aussi à fenêtres).

In [ ]:
from __future__ import division
import holoviews as hv
hv.extension('bokeh')
import param
import holoviews as hv,panel as pn,param
from holoviews.streams import Pipe
import time
import pandas as pd
import panel as pn
from panel.pane import LaTeX
from scipy.io.wavfile import read
from IPython.display import Audio
import requests
from io import BytesIO
from PIL import Image
import shutil
from urllib.request import urlopen
import io
import scipy.io as sio

import numpy as np
import scipy as scp
import pylab as pyl
from matplotlib import cm
import matplotlib.pyplot as plt
from scipy import fftpack
from matplotlib.pyplot import imshow as imageplot
from mpl_toolkits.mplot3d import Axes3D
import wave
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def f1(x):
    y=np.mod(x+np.pi,2*np.pi)-np.pi
    return y**2

In [ ]:
def SommesPartiellesExo1(n):
    N=10000
    t=np.linspace(-3*np.pi,3*np.pi,N)
    y1=f1(t)
    C1=hv.Curve((t,y1),label='Courbe de la fonction f').opts(width=650)
    z=np.zeros(N)+np.pi**2/3
    for k in np.arange(1,n):
        z=z+4*(((-1)**k)/k**2)*np.cos(t*k)
    return C1*hv.Curve((t,z),label='Sommes partielles de Fourier')   

In [ ]:
z=SommesPartiellesExo1(20)
z

In [ ]:
def f5(x):
    y=np.mod(x,2*np.pi)
    if y<np.pi:
        z=1
    else:
        z=0
    return z
vf5=np.vectorize(f5)

In [ ]:
def SommesPartiellesExo5(n):
    N=10000
    t=np.linspace(-3*np.pi,3*np.pi,N)
    y5=vf5(t)
    C5=hv.Curve((t,y5),label='Courbe de la fonction f').opts(width=650)
    z=np.zeros(N)+0.5
    for k in np.arange(1,n):
        z=z-(((-1)**k-1)/(np.pi*k))*np.sin(t*k)
    return C5*hv.Curve((t,z),label='Sommes partielles de Fourier')   

In [ ]:
z=SommesPartiellesExo5(40)
z

In [ ]:
def f6(x):
    y=np.mod(x,2*np.pi)
    if y<np.pi:
        z=1
    else:
        z=-1
    return z
vf6=np.vectorize(f6)

In [ ]:
def SommesPartiellesExo6(n):
    N=10000
    t=np.linspace(-3*np.pi,3*np.pi,N)
    y6=vf6(t)
    C6=hv.Curve((t,y6),label='Courbe de la fonction f').opts(width=650)
    z=np.zeros(N)
    for k in np.arange(1,n):
        z=z-2*(((-1)**k-1)/(np.pi*k))*np.sin(t*k)
    return C6*hv.Curve((t,z),label='Sommes partielles de Fourier') 

In [ ]:
z=SommesPartiellesExo6(100)
z

In [ ]:
def SommesPartiellesDiscretesExo6(n):
    N=10000
    t=np.linspace(0,2*np.pi,N)
    y=vf6(t)
    C=hv.Curve((t,y),label='Courbe de la fonction f sur [0;2 pi]').opts(width=650)
    z=np.zeros(N,dtype=complex)
    temp=fftpack.fft(y)
    z[0:n]=temp[0:n]
    z[N-n:N]=temp[N-n:N]
    res=np.real(fftpack.ifft(z))
    return C*hv.Curve((t,res),label='Approximation de Fourier Discrete').opts(width=650)

z=SommesPartiellesDiscretesExo6(100)
z

# Effets de Gibbs.

L'effet de Gibbs est l'effet de rebond que l'on observe tant en approximation continue que discrète au niveau des discontinuité. Ce rebond ne diminue pas en amplitude, même pour les grandes valeurs de $n$... On peut observer qu'il reste au dessus de la valeur 1.17...
Dans l'exercice 6, on montre qu'il vaut pour un $n$ donné :
\begin{equation}
f_n(t_n)=\frac{2}{\pi}\frac{1}{n+1}\sum_{k=0}^n\frac{\sin\left(\frac{\pi(2k+1)}{2n+2}\right)}{\frac{2k+1}{2n+2}}
\end{equation}
que l'on peut calculer par les commandes suivantes :

In [ ]:
def Gibbs(n):
    z=0
    for k in np.arange(0,n):
        xk=np.pi*(2*k+1)/(2*n+2)
        z=z+np.sin(xk)/xk
    return z*2/(n+1)
print(Gibbs(400))

On peut observer que cette somme converge vers une intégrale en remarquant que la somme peut être vue comme une approximation par une somme d'aires de rectangles c'est-à-dire comme une somme de Riemmann. 

Si on représente graphiquement le vecteur dont la kieme composante est égale à 
$\frac{\sin\left(\frac{\pi(2k+1)}{2n+2}\right)}{\frac{2k+1}{2n+2}}$, on obtient le graphe suivant :

In [ ]:
def VecGibbs(n):
    z=np.zeros(n)   
    a=2/(n+1)
    for k in np.arange(0,n):
        xk=np.pi*(2*k+1)/(2*n+2)
        z[k]=np.sin(xk)/xk
    return z
test=VecGibbs(100)
hv.Bars(test).opts(width=650,xaxis=None)

On peut remarquer que la somme associé converge vers l'intégrale d'une fonction... en effet 


In [ ]:
n=100
test=VecGibbs(n)
hv.Bars(test).opts(width=650,xaxis=None)
t=np.linspace(0.0001,np.pi,n)
y=np.sin(t)/t
Cu=hv.Curve(y,label='Fontion f(x)=sin(x)/x entre 0 et pi').opts(color='red')
hv.Bars(test).opts(margin=10,width=650,xaxis=None)*Cu

# Echantillonnage.

L'objectif de cette section est de comprendre le théorème d'échantillonnage de Shannon présenté dans le poly de cours. Ce théorème qui explique comment on peut reconstruire un signal analogique à partir d'un signal numérique (ou digital) ne peut être directement illustré via un notebook dans ma mesure où tous les signaux considérés sont numériques, ce sotn tous des vecteurs.

Nous allons essayer de comprendre ici comment et à quelle condition on peut reconstruire un signal numérique plus ou moins parfaitement à partir d'une version sous-échantillonnée.

In [ ]:
local=0
if local:
    S3=np.load('Piece.npy')
    S4=np.load('Blocks.npy')   
else:
    url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Blocks.npy'
    response = requests.get(url, stream=True)
    with open('blocks.npy', 'wb') as fin:
        shutil.copyfileobj(response.raw, fin)
    S4=res=np.load('blocks.npy')
    url='https://plmlab.math.cnrs.fr/dossal/optimisationpourlimage/raw/master/img/Piece.npy'
    response = requests.get(url, stream=True)
    with open('piece.npy', 'wb') as fin:
        shutil.copyfileobj(response.raw, fin)
    S3=res=np.load('piece.npy')
#pn.Row(hv.Curve(S3),hv.Curve(S4))
def Gaussienne(N,sigma2,b):
    x = np.linspace(-b, b, N)
    y=np.exp(-np.abs(x)*np.abs(x)/sigma2)
    return y
gau=Gaussienne(1024,1,10)
Fgau=fftpack.fft(gau)
SignauxRef= {"Piece" : S3,"Blocks" : S4,"Gaussienne": gau}

Autrement dit : est il possible de reconstruire correctement le vecteur du haut sur 1024 points à partir de celui du bas échantillonné à 1 point sur 4 ?

In [ ]:
S=SignauxRef["Piece"]
Sechan=S[0:1024:4]
pn.Column(hv.Curve(S,label='original vector').opts(width=700),hv.Curve(Sechan,label='sampled vector').opts(width=700))

Une idée naïve serait de recopier chacune des valeurs 4 fois ou de faire une interpolation linéaire. Or aucune de ces méthodes ne donnent de résultats satisfaisants.

Une manière d'effectuer cette reconstruction est de comprendre la manière dont l'échantillonnage agit sur un vecteur dans le domaine de Fourier. On affiche ici les modules des transformées de Fourier des deux signaux... l'un est sur 1024 points, le second sur 256 :

In [ ]:
FS=np.abs(fftpack.fft(S))
FSechan=np.abs(fftpack.fft(Sechan))
pn.Column(hv.Curve(FS,label='Module dof the fft of the original vector').opts(width=700)\
          ,hv.Curve(FSechan,label='Module dof the fft of the sampled vector').opts(width=700))

Il n'est pas possible de comparer directement les fft (ou les fréquences) de ces deux vecteurs car ils ne sont pas de la même dimension. Il faut toujours garder à l'esprit que ces suites nesont que des représentants d'une période d'une suite infinie périodique.
Pour mieux comprendre l'effet d'un échantillonage sur un vecteur, il est préférable de l'échantillonner en conservant un vecteur de même taille que le vecteur original et de mettre les coefficients qu'on ne sélectionne pas à 0 :

In [ ]:
Sechan2=S*0
Sechan2[0:1024:4]=Sechan
FSechan2=np.abs(fftpack.fft(Sechan2))
pn.Row(pn.Column(hv.Curve(S),hv.Curve(Sechan2),hv.Curve(Sechan)),\
      pn.Column(hv.Curve(FS),hv.Curve(FSechan2),hv.Curve(FSechan)))

On peut faire plusieurs observations à partir de ces graphiques :
Si la transformée de Fourier de la dernière ligne correspond à une seule période de la suite infinie des $\hat f[k]$, la seconde ligne correspond à 4 périodes de cette même suite.
Comme on peut le vérifier aisément en zoomant sur le les 256 premiers coefficients de la fft de la seconde ligne ou en comparant les deux vecteurs en les superposant.

In [ ]:
hv.Curve(FSechan,label='FSechan')\
*hv.Curve(FSechan2[0:256],label='256 premiers coefficients de FSechan2').opts(width=700)

Une seconde remarque qu'on peut faire est que les coefficients de Fourier discret du signal échantillonnés... sont obtenus à partir des coefficients du signal original en les périodisant. Ainsi le vecteur de droite, FFT du signal échantillonné peut être obtenu en sommant des versions décalées de la FFT du signal original. Démonstration :

In [ ]:
FSe=fftpack.fft(S)
FS1=np.roll(FSe,256)
FS2=np.roll(FSe,512)
FS3=np.roll(FSe,768)
hv.Curve(np.abs(FS))*hv.Curve(np.abs(FS1))*hv.Curve(np.abs(FS2))*hv.Curve(np.abs(FS3))
Fper=(FSe+FS1+FS2+FS3)/4
C=hv.Curve(np.abs(FSe))*hv.Curve(np.abs(FS1))\
*hv.Curve(np.abs(FS2))*hv.Curve(np.abs(FS3)).opts(width=800,title='spectre périodisé')
Frec=np.real(fftpack.ifft(Fper))

pn.Column(C,hv.Curve(Frec,label='Signal obtenu par périodisation de la FFT').opts(width=800),
          hv.Curve(Sechan2,label='Signal echantillonné').opts(width=800))

Une manière de comprendre ce lien entre échantillonnage du signal et périodisation est de voir que l'échantillonnage correspond à une multiplication par un peigne de Dirac.

In [ ]:
P=np.zeros(1024)
P[0:1024:4]=np.ones(256)
Echan3=S*P
pn.Column(hv.Curve(S).opts(width=800),hv.Curve(P).opts(width=800),hv.Curve(Echan3).opts(width=800))

Ainsi l'échantillonnage va se traduire par une convolution dans le domaine de Fourier. une convolution par la transformée de Fourier d'un peigne de Dirac. Or la transformée de Fourier d'un Peigne de Dirac est un peigne de Dirac... et la convolution par un peigne de Dirac est exactement une périodisation. Par exemple si on échantillonne en ne prenant un point sur 4, on périodise la FFT 4 fois.   

In [ ]:
pn.Column(hv.Curve(FS,label='FFT du signal original')\
          .opts(width=800),hv.Curve(FSechan2,label='FFT du signal échantillonné').opts(width=800))

# Reconstruire un signal à partir d'un signal sous échantillonné. 

L'idée de l'algorithme de reconstruction est assez simple : On élimine les réplications du spectre (dans le domaine de Fourier) en effectuant une multiplication par une indicatrice dans le domaine de Fourier :

In [ ]:
ind=np.zeros(1024)
ind[0:128]=np.ones(128)
ind[1024-128:1024]=np.ones(128)
M=np.max(FSechan2)
FSrec=np.real(FSechan2*ind)
pn.Column(hv.Curve(FSechan2,label='FFT du signal échantillonné')*hv.Curve(M*ind).opts(width=800),\
          hv.Curve(M*ind).opts(color='red')*hv.Curve(FSrec,label='FFT recosntruite par filtrage').opts(width=800,color='black'))

On reconstruit alors le signal en effectuant une transformée de Fourier inverse :

In [ ]:
temp=fftpack.fft(Sechan2)*ind
Srec=np.real(fftpack.ifft(temp))*4
hv.Curve(S,label='Signal original')*\
hv.Curve(Srec,label='Signal reconstruit à partir du signal sous-échantillonné').opts(width=800)


On peut observer que le signal est plutôt bien reconstruit sur les parties régulières mais certaines hautes fréquences manquent ce qui induit des effets de Gibbs.

En résumé l'échantillonnage induit des réplications périodiques du spectre à des fréquences qui sont des multiples de la fréquence d'échantillonage. Par exemple, si on échantillonne tous les 4 points et qu'on considère un signal sur 1024 points on réplique le spectre en ajoutant des "fantômes" tous les 1024/4=256 points. 
Si on échantillonne tous les 8 points on crée encore plus de fantômes du spectre :

In [ ]:
Sechan8=0*S
Sechan8[0:1024:8]=S[0:1024:8]
FS8=np.abs(fftpack.fft((Sechan8)))
pn.Column(hv.Curve(FS,label='FFT du signal original').opts(width=800),\
         hv.Curve(FS8,label='FFT du signal échantillonné tous les 8 points').opts(width=800))

Pour reconstruire le signal, on le multplie dans le domaine de Fourier par une indicatrice et on effectue une transformée de Fourier inverse. Une telle reconstruction est équivalente à effectuer une convolution dans le domaine spatial (ou temporel) par un sinus cardinal.

In [ ]:
ind8=np.zeros(1024)
ind8[0:64]=np.ones(64)
ind8[1024-64:1024]=np.ones(64)
M=np.max(FS8)
temp=np.real(fftpack.ifft(fftpack.fft(Sechan8)*ind8))*8
pn.Column(hv.Curve(FS8,label='FFT du signal échantillonné tous les 8 points').opts(width=800)\
          *hv.Curve(M*ind8)*hv.VLine(128).opts(color='black'),\
          hv.Curve(S,label='Sinal original')*hv.Curve(temp,label='Signal reconstruit').opts(width=800))

Les questions qui se posent alors sont : 

1) Pourquoi la reconstruction n'est pas toujours parfaite ?

2) A quelle condition l'est elle ?

3) Si cette condition n'est pas vérifiée, peut on améliorer le procédé d'échantillonnage ?

La réponse à la première question est simple : 
L'échantillonnage en périodisant le spectre a pour effet de mélanger les hautes et les basses fréquences c'est ce qu'on appelle le repliement spectral ou l'aliasing en anglais. Ainsi même les basses fréquences du signal échantillonné peuvent être différentes de celles du signal original. De plus, le procédé de reconstruction coupe les hautes fréquences. Ainsi les hautes fréquences posent un problème double :
on ne peut pas les recosntruire et en plus elles peuvent altérer les basses fréquences.

Mais de quelles hautes fréquences parle-t-on ? 
Si on considère un signal sur N points et qu'on échantillonne le signal tous les d points, les "hautes fréquences" qui poseront problème pour la reconstruction seront celles dont les indices sont entre $\frac{N}{2d}$ et 
$N-\frac{N}{2d}$. Si on note $Fe=\frac{N}{d}$ la fréquence d'échantillonnage, le procédé de reconstruction du signal à partir de sa version sous-échantillonnée à une fréquence $Fe$ est exacte si pour toutes les fréquences $k>F_{max}=2Fe$ on a $\hat f[k]=0$. C'est-à-dire si la transformée de Fourier du signal original est à support dans $[-F_{max},F_{max}]$ avec $F_{max}>2Fe$.  



In [ ]:
class Echansig(param.Parameterized):
    Signal = param.ObjectSelector(default="Gaussienne",objects=SignauxRef.keys())
    logEchan = param.Integer(4,bounds=(1,7))
    def view(self):
        S=SignauxRef[self.Signal]
        gau=S
        N=1024
        d=2**self.logEchan
        Sechan=0*S
        Sechan[0:N:d]=S[0:N:d]
        FSechan=fftpack.fft(Sechan)*d
        FS=fftpack.fft(S)
        ind=np.zeros(N)
        Fe=int(N/(2*d))
        ind[0:Fe]=np.ones(Fe)
        ind[N-Fe:N]=np.ones(Fe)
        Frec=FSechan*ind
        Srec=np.real(fftpack.ifft(Frec))
        C1=hv.Curve(S,label='Signal original').opts(width=800)
        C2=hv.Curve(Sechan,label='Signal échantillonné').opts(width=800)
        C3=hv.Curve(Srec,label='Signal reconstruit').opts(width=800,color='black')
        C4=hv.Curve(np.abs(FS),label='FFT du Signal original').opts(width=800)
        C5=hv.Curve(np.abs(FSechan),label='FFT du Signal echantillonné').opts(width=800)
        C6=hv.Curve(np.abs(Frec),label='FFT du Signal reconstruit').opts(width=800,color='black')
        return pn.Column(C1*C2*C3,C4*C5*C6*hv.VLine(int(1024/d)).opts(color='green'))


In [ ]:
Es= Echansig()
pn.Column(Es.param,Es.view)

On peut limiter le repliement spectral des hautes fréquences sur les basses fréquences en les coupant avant l'échantillonnage, c'est-à-dire en filtrant le signal avant l'échantillonnage. Cette opération de filtrage est appelé antialising. Ainsi pour échantillonner un signal en utilisant un filtre anti-aliasing :

1) On filtre le signal en supprimant toutes les fréquences supérieurs à $2Fe$.

2) On échantillone ensuite le signal.

3) On reconstruit ensuite avec la même procédure que précedemment.

Dans l'exemple ci dessous on voit le gain en terme de norme et visuel d'un filtre antialising.

In [ ]:
S=SignauxRef["Blocks"]
N=1024
Fe=64
d=int(N/(2*Fe))
ind=np.zeros(N)
ind[0:Fe]=np.ones(Fe)
ind[N-Fe:N]=np.ones(Fe)
Sechan=np.zeros(N)
Sechan[0:N:d]=S[0:N:d]
FS=fftpack.fft(S)
FSalias=FS*ind
temp=np.real(fftpack.ifft(FSalias))
Sechan2=np.zeros(N)
Sechan2[0:N:d]=temp[0:N:d]
FSechan=fftpack.fft(Sechan)
hv.Curve(S,label='Signal original')*hv.Curve(Sechan,label='Signal échantillonné sans filtre antiliasing')\
*hv.Curve(Sechan2,label='Signal échantillonné avec filtre antiliasing').opts(width=800)

In [ ]:
Srec1=np.real(fftpack.ifft(FSechan*ind))*d
Srec2=np.real(fftpack.ifft(fftpack.fft(Sechan2)*ind))*d
hv.Curve(S,label='Signal original').opts(width=800)*\
hv.Curve(Srec1,label='Reconstruction sans filtrage antialiasing').opts(width=800)*\
hv.Curve(Srec2,label='Reconstruction avec filtrage antialiasing').opts(width=800)

L'erreur que l'on commet sans filtre antialiasing est environ deux fois supérieur à ce que l'on commet quand on filtre préalablement le signal. Pourquoi ?

In [ ]:
print(np.sum(np.abs((S-Srec1)**2)),np.sum(np.abs((S-Srec2)**2)))